In [1]:
import pickle
import sys
sys.path.append("../")

import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.metrics import mean_squared_error
from math import sqrt
from surprise import accuracy
from reco_utils.dataset.python_splitters import python_stratified_split
from reco_utils.evaluation.python_evaluation import rmse
from reco_utils.recommender.sar import SAR
from reco_utils.recommender.ncf.ncf_singlenode import NCF

In [2]:
tf.logging.set_verbosity(tf.logging.ERROR)

In [3]:
test = pd.read_csv('../Data/test.csv', names=['user_id', 'profile_id', 'rating'])

In [4]:
test_knn = pd.read_csv('../Data/test.csv', names=['user_id', 'profile_id', 'rating'])
test_knn['user_id'] = test_knn['user_id'].astype(str)
test_knn['profile_id'] = test_knn['profile_id'].astype(str)

tuples = [tuple(x) for x in test_knn.to_numpy()]

In [5]:
f = open('../kNN/model/KNN.pickle','rb') 
loaded_model = pickle.load(f)
f.close()

In [6]:
predictions = loaded_model.test(tuples)

In [7]:
predictions_knn = [el.est for el in predictions]

In [8]:
test['pred_knn'] = predictions_knn

In [9]:
f = open('../SAR/model/SAR.pickle','rb') 
loaded_sar = pickle.load(f)
f.close()

In [10]:
predictions = loaded_sar.predict(test, normalize=True)

../reco_utils/recommender/sar/sar_singlenode.py:333: RuntimeWarning: invalid value encountered in true_divide
  self.unity_user_affinity[user_ids, :].dot(self.item_similarity),
Items found in test not seen during training, new items will have score of 0


In [11]:
predictions = predictions.replace([np.inf, -np.inf], 0)

In [12]:
test['pred_sar'] = predictions['prediction']

In [13]:
model = NCF (
    n_users=2000, 
    n_items=1531,
    model_type="NeuMF",
    n_factors=8,
    layer_sizes=[32,16,8]
)

In [14]:
f = open('../NCF/model/user2id_embedding.pickle','rb') 
user2id_embedding = pickle.load(f)
f.close()

f = open('../NCF/model/item2id_embedding.pickle','rb') 
item2id_embedding = pickle.load(f)
f.close()

f = open('../NCF/model/id2user_embedding.pickle','rb') 
id2user_embedding = pickle.load(f)
f.close()

f = open('../NCF/model/id2item_embedding.pickle','rb') 
id2item_embedding = pickle.load(f)
f.close()

In [15]:
model.load(neumf_dir='../NCF/model')
model.user2id = user2id_embedding
model.item2id = item2id_embedding
model.id2user = id2user_embedding
model.id2item = id2item_embedding

In [16]:
test['predictions_ncf'] = test.apply(lambda x: model.predict(x.user_id, x.profile_id), axis=1)
test['predictions_ncf'] *= 10

rms = sqrt(mean_squared_error(test['rating'].values, test['predictions_ncf'].values))
print(rms)

In [18]:
# ensemble knn and sar

test['ensemble'] = test['pred_knn'] * 0.6 + test['pred_sar'] * 0.4

rms = sqrt(mean_squared_error(test['rating'].values, test['ensemble'].values))

print(rms)

2.1120785294155144


In [19]:
# ensemble knn and ncf

test['ensemble'] = test['pred_knn'] * 0.6 + test['predictions_ncf'] * 0.4

rms = sqrt(mean_squared_error(test['rating'].values, test['ensemble'].values))

print(rms)

2.5456876498682486


In [20]:
# ensemble sar and ncf

test['ensemble'] = test['pred_sar'] * 0.6 + test['predictions_ncf'] * 0.4

rms = sqrt(mean_squared_error(test['rating'].values, test['ensemble'].values))

print(rms)

3.102747699270003
